In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from __future__ import print_function
from statsmodels.compat import lzip
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
from sklearn.linear_model import LinearRegression

In [2]:
from statsmodels.graphics.regressionplots import plot_fit
from statsmodels.graphics.regressionplots import plot_leverage_resid2
from statsmodels.graphics.regressionplots import plot_regress_exog
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.pipeline import make_pipeline

In [3]:
from sklearn.metrics import r2_score, mean_squared_error
linear_regression = linear_model.LinearRegression(normalize=False, fit_intercept=True)
def r2_est(X,y):
    return r2_score(y, linear_regression.fit(X,y).predict(X))

In [4]:
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.cross_validation import cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import make_scorer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import RFECV
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import Lasso
from sklearn.grid_search import RandomizedSearchCV
from scipy.stats import expon

## Data Loading and Preparation (time series)

In [6]:
cal = pd.read_csv('Datasources/inside_airbnb/calendar.csv')

In [8]:
cal['price'] = cal['price'].astype(str).map(lambda x: x.lstrip('$'))
cal['price'] = cal['price'].apply(pd.to_numeric, errors='coerce')
cal['price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [9]:
cal['date'] = pd.to_datetime(cal['date'])
cal['month'] = cal['date'].apply(lambda x: x.month)
cal['day'] = cal['date'].apply(lambda x: x.day)

In [10]:
cal['day_of_week'] = cal['date'].dt.weekday_name

In [11]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

In [12]:
cl = calendar()
holidays = cl.holidays(start=cal['date'].min(), end=cal['date'].max())

In [13]:
cal['holiday'] = cal['date'].isin(holidays)

In [14]:
c = cal.loc[cal.available!='f']
c = c[['listing_id','date','price','month','day_of_week','holiday']]

In [15]:
c_hol = c[c['holiday']==True]
c_wke = c[(c['holiday']==False)&((c['day_of_week']=='Sunday')|(c['day_of_week']=='Saturday'))]
c_wkd = c[(~c.isin(c_hol['date']))&(~c.isin(c_wke['date']))]

In [16]:
mean_price_hol = c_hol.groupby(by='listing_id')['price'].mean()
mean_price_wke = c_wke.groupby(by='listing_id')['price'].mean()
mean_price_wkd = c_wkd.groupby(by='listing_id')['price'].mean()

In [17]:
mean_price_hol = pd.DataFrame(mean_price_hol)
mean_price_wke = pd.DataFrame(mean_price_wke)
mean_price_wkd = pd.DataFrame(mean_price_wkd)

In [18]:
mean_price_hol = mean_price_hol.reset_index()
mean_price_wke = mean_price_wke.reset_index()
mean_price_wkd = mean_price_wkd.reset_index()

In [19]:
listings = pd.read_csv('Datasources/inside_airbnb/listings.csv')

In [20]:
master_hol = listings.merge(mean_price_hol, how='inner', left_on='id', right_on='listing_id')
master_wke = listings.merge(mean_price_wke, how='inner', left_on='id', right_on='listing_id')
master_wkd = listings.merge(mean_price_wkd, how='inner', left_on='id', right_on='listing_id')

## Feature Creation (time series)

In [21]:
master_hol['host_response_rate'] = master_hol['host_response_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_hol['host_response_rate'] = master_hol['host_response_rate'].apply(pd.to_numeric, errors='coerce')
master_hol['host_response_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['host_acceptance_rate'] = master_hol['host_acceptance_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_hol['host_acceptance_rate'] = master_hol['host_acceptance_rate'].apply(pd.to_numeric, errors='coerce')
master_hol['host_acceptance_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['extra_people'] = master_hol['extra_people'].astype(str).map(lambda x: x.lstrip('$'))
master_hol['extra_people'] = master_hol['extra_people'].apply(pd.to_numeric, errors='coerce')
master_hol['extra_people'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['weekly_price'] = master_hol['weekly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_hol['weekly_price'] = master_hol['weekly_price'].apply(pd.to_numeric, errors='coerce')
master_hol['weekly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['monthly_price'] = master_hol['monthly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_hol['monthly_price'] = master_hol['monthly_price'].apply(pd.to_numeric, errors='coerce')
master_hol['monthly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['security_deposit'] = master_hol['security_deposit'].astype(str).map(lambda x: x.lstrip('$'))
master_hol['security_deposit'] = master_hol['security_deposit'].apply(pd.to_numeric, errors='coerce')
master_hol['security_deposit'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['cleaning_fee'] = master_hol['cleaning_fee'].astype(str).map(lambda x: x.lstrip('$'))
master_hol['cleaning_fee'] = master_hol['cleaning_fee'].apply(pd.to_numeric, errors='coerce')
master_hol['cleaning_fee'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [22]:
master_wke['host_response_rate'] = master_wke['host_response_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_wke['host_response_rate'] = master_wke['host_response_rate'].apply(pd.to_numeric, errors='coerce')
master_wke['host_response_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['host_acceptance_rate'] = master_wke['host_acceptance_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_wke['host_acceptance_rate'] = master_wke['host_acceptance_rate'].apply(pd.to_numeric, errors='coerce')
master_wke['host_acceptance_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['extra_people'] = master_wke['extra_people'].astype(str).map(lambda x: x.lstrip('$'))
master_wke['extra_people'] = master_wke['extra_people'].apply(pd.to_numeric, errors='coerce')
master_wke['extra_people'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['weekly_price'] = master_wke['weekly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_wke['weekly_price'] = master_wke['weekly_price'].apply(pd.to_numeric, errors='coerce')
master_wke['weekly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['monthly_price'] = master_wke['monthly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_wke['monthly_price'] = master_wke['monthly_price'].apply(pd.to_numeric, errors='coerce')
master_wke['monthly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['security_deposit'] = master_wke['security_deposit'].astype(str).map(lambda x: x.lstrip('$'))
master_wke['security_deposit'] = master_wke['security_deposit'].apply(pd.to_numeric, errors='coerce')
master_wke['security_deposit'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['cleaning_fee'] = master_wke['cleaning_fee'].astype(str).map(lambda x: x.lstrip('$'))
master_wke['cleaning_fee'] = master_wke['cleaning_fee'].apply(pd.to_numeric, errors='coerce')
master_wke['cleaning_fee'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [23]:
master_wkd['host_response_rate'] = master_wkd['host_response_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_wkd['host_response_rate'] = master_wkd['host_response_rate'].apply(pd.to_numeric, errors='coerce')
master_wkd['host_response_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['host_acceptance_rate'] = master_wkd['host_acceptance_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_wkd['host_acceptance_rate'] = master_wkd['host_acceptance_rate'].apply(pd.to_numeric, errors='coerce')
master_wkd['host_acceptance_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['extra_people'] = master_wkd['extra_people'].astype(str).map(lambda x: x.lstrip('$'))
master_wkd['extra_people'] = master_wkd['extra_people'].apply(pd.to_numeric, errors='coerce')
master_wkd['extra_people'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['weekly_price'] = master_wkd['weekly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_wkd['weekly_price'] = master_wkd['weekly_price'].apply(pd.to_numeric, errors='coerce')
master_wkd['weekly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['monthly_price'] = master_wkd['monthly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_wkd['monthly_price'] = master_wkd['monthly_price'].apply(pd.to_numeric, errors='coerce')
master_wkd['monthly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['security_deposit'] = master_wkd['security_deposit'].astype(str).map(lambda x: x.lstrip('$'))
master_wkd['security_deposit'] = master_wkd['security_deposit'].apply(pd.to_numeric, errors='coerce')
master_wkd['security_deposit'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['cleaning_fee'] = master_wkd['cleaning_fee'].astype(str).map(lambda x: x.lstrip('$'))
master_wkd['cleaning_fee'] = master_wkd['cleaning_fee'].apply(pd.to_numeric, errors='coerce')
master_wkd['cleaning_fee'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

### Amenities Feature Creation

In [27]:
def string_to_set(x):
    c = set()
    for w in x[1:-1].split(","):
        c.add(w)
        
    return c

In [28]:
master_hol['amenities_set'] = master_hol['amenities'].map(string_to_set)
master_wke['amenities_set'] = master_wke['amenities'].map(string_to_set)
master_wkd['amenities_set'] = master_wkd['amenities'].map(string_to_set)

In [29]:
def has_amenity(x, amen_):
    if amen_ in x:
        return 1
    pass

In [31]:
all_amenities = set()

for idx in listings['amenities'].map(string_to_set).index:
    all_amenities = all_amenities.union(listings['amenities'].map(string_to_set)[idx])

all_amenities

{'',
 '"24-Hour Check-in"',
 '"Air Conditioning"',
 '"Buzzer/Wireless Intercom"',
 '"Cable TV"',
 '"Carbon Monoxide Detector"',
 '"Elevator in Building"',
 '"Family/Kid Friendly"',
 '"Fire Extinguisher"',
 '"First Aid Kit"',
 '"Free Parking on Premises"',
 '"Hair Dryer"',
 '"Hot Tub"',
 '"Indoor Fireplace"',
 '"Laptop Friendly Workspace"',
 '"Lock on Bedroom Door"',
 '"Other pet(s)"',
 '"Pets Allowed"',
 '"Pets live on this property"',
 '"Safety Card"',
 '"Smoke Detector"',
 '"Smoking Allowed"',
 '"Suitable for Events"',
 '"Wheelchair Accessible"',
 '"Wireless Internet"',
 '"translation missing: en.hosting_amenity_49"',
 '"translation missing: en.hosting_amenity_50"',
 'Breakfast',
 'Cat(s)',
 'Dog(s)',
 'Doorman',
 'Dryer',
 'Essentials',
 'Gym',
 'Hangers',
 'Heating',
 'Internet',
 'Iron',
 'Kitchen',
 'Pool',
 'Shampoo',
 'TV',
 'Washer'}

In [32]:
for amen in all_amenities:
    master_hol['has' + amen] = 0
    master_wke['has' + amen] = 0
    master_wkd['has' + amen] = 0
    master_hol['has' + amen] = master_hol['amenities_set'].map(lambda x: has_amenity(x, amen))
    master_wke['has' + amen] = master_wke['amenities_set'].map(lambda x: has_amenity(x, amen))
    master_wkd['has' + amen] = master_wkd['amenities_set'].map(lambda x: has_amenity(x, amen))

In [33]:
has_amenties_list = []
for amen in all_amenities:
    has_amenties_list.append('has' + amen)

In [34]:
master_hol[has_amenties_list] = master_hol[has_amenties_list].fillna(0)
master_wke[has_amenties_list] = master_wke[has_amenties_list].fillna(0)
master_wkd[has_amenties_list] = master_wkd[has_amenties_list].fillna(0)

In [35]:
target_hol = master_hol['price_y']
target_wke = master_wke['price_y']
target_wkd = master_wkd['price_y']

In [41]:
X_hol = master_hol.select_dtypes(include=['float64', 'int64'])
X_wke = master_wke.select_dtypes(include=['float64', 'int64'])
X_wkd = master_wkd.select_dtypes(include=['float64', 'int64'])

## Modeling (time series)